import libraries

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

read template and scanned image

In [ ]:
#reference image
refFilename = "form.jpg"
print("Reading reference image : ", refFilename)
im1 = cv2.imread(refFilename, cv2.IMREAD_COLOR)
im1 = cv2.cvtColor(im1, cv2.COLOR_BGR2RGB)

#image to be aligned
imFilename = "scanned-image.jpg"
print("Reading image to align : ", imFilename)
im2 = cv2.imread(imFilename, cv2.IMREAD_COLOR)
im2 = cv2.cvtColor(im2, cv2.COLOR_BGR2RGB)


display images

In [ ]:
plt.figure(figsize=(10,10))
plt.subplot(1,2,1); plt.axis('off'); plt.imshow(im1); plt.title('Reference image')
plt.subplot(1,2,2); plt.axis('off'); plt.imshow(im2); plt.title('Image to align')

find keypoints in both images

In [ ]:
im1_gray = cv2.cvtColor(im1, cv2.COLOR_BGR2GRAY)
im2_gray = cv2.cvtColor(im2, cv2.COLOR_BGR2GRAY)

#detect orb features and compute descriptors
MAX_NUM_FEATURES = 500
orb = cv2.ORB_create(MAX_NUM_FEATURES)
keypoints1, descriptors1 = orb.detectAndCompute(im1_gray, None)
keypoints2, descriptors2 = orb.detectAndCompute(im2_gray, None)

#display
im1_display = cv2.drawKeypoints(im1, keypoints1, outImage = np.array([]), color = (0,255,0), flags = cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
im2_display = cv2.drawKeypoints(im2, keypoints2, outImage = np.array([]), color = (0,255,0), flags = cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

In [ ]:
plt.figure(figsize=(20,10))
plt.subplot (1,2,1); plt.axis('off'); plt.imshow(im1_display); plt.title('Reference image')
plt.subplot (1,2,2); plt.axis('off'); plt.imshow(im2_display); plt.title('Image to align')

match keypoints in the two images

In [ ]:
#match 
matcher = cv2.DescriptorMatcher_create(cv2.DESCRIPTOR_MATCHER_BRUTEFORCE_HAMMING)
matches = matcher.match(descriptors1, descriptors2, None)

#sort matches by score
matches.sort(key = lambda x: x.distance, reverse=False)

#remove not so good matches
numGoodMatches = int(len(matches)*0.1)
matches = matches[:numGoodMatches]

In [ ]:
im_matches = cv2.drawMatches(im1, keypoints1, im2, keypoints2, matches, None)
plt.figure(figsize=(40,10))
plt.imshow(im_matches); plt.axis('off'); plt.title('original form')

find homography

In [ ]:
points1 = np.zeros((len(matches), 2), dtype=np.float32)
points2 = np.zeros((len(matches), 2), dtype=np.float32)

for i, match in enumerate(matches):
    points1[i, :] = keypoints1[match.queryIdx].pt
    points2[i, :] = keypoints2[match.trainIdx].pt

#find homography
h, mask = cv2.findHomography(points1, points2, cv2.RANSAC)

warp image

In [ ]:
height, weight, channels = im1.shape
im2_registered = cv2.warpPerspective(im2, h, (weight, height))

#display results
plt.figure(figsize=(20,10))
plt.subplot (1,2,1); plt.axis('off'); plt.imshow(im1); plt.title('Reference image')
plt.subplot (1,2,2); plt.axis('off'); plt.imshow(im2_registered); plt.title('Registered image')
